In [1]:
!pip3 install pyomo
import pyomo
from pyomo.environ import *
from pyomo.opt import SolverFactory

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 30.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 2.4 MB/s eta 0:00:00


In [2]:

!apt install glpk-utils
!pip install glpk

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
Suggested packages:
  libiodbc2-dev
The following NEW packages will be installed:
  glpk-utils libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
0 upgraded, 5 newly installed, 0 to remove and 45 not upgraded.
Need to get 625 kB of archives.
After this operation, 2,158 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 libsuitesparseconfig5 amd64 1:5.10.1+dfsg-4build1 [10.4 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libamd2 amd64 1:5.10.1+dfsg-4build1 [21.6 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/main amd64 libcolamd2 amd64 1:5.10.1+dfsg-4build1 [18.0 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libglpk40 amd64 5.0-1 [361 kB]
Get:5 http://archive.ubuntu.com/ubuntu jammy/universe amd64 glpk-ut

In [3]:
#Parameters and Input Data
# Define demand for each customer
d = {1: 80, 2: 270, 3: 250, 4: 160, 5: 180}
# Define a list of customers
I = list(d.keys())

# Define the maximum amounts and activation costs for each site
M = {1: 500, 2: 500, 3: 500}
f = {1: 1000, 2: 1000, 3: 1000}
# Define a list of facility sites
J = list(M.keys())

# Define transportation costs
c = {
    (1, 1): 4,  (1, 2): 6,  (1, 3): 9,
    (2, 1): 5,  (2, 2): 4,  (2, 3): 7,
    (3, 1): 6,  (3, 2): 3,  (3, 3): 4,
    (4, 1): 8,  (4, 2): 5,  (4, 3): 3,
    (5, 1): 10, (5, 2): 8,  (5, 3): 4,
}

In [4]:
#Model
model = ConcreteModel()

model.I = Set(initialize=I)
model.J = Set(initialize=J)

#Variables
model.x = Var(model.I, model.J, domain=NonNegativeReals)  # Amount serviced from facility to demand point
model.y = Var(model.J, domain=Binary)

#Objective
model.obj = Objective(
        expr=sum(f[j] * model.y[j] for j in model.J) +
             sum(c[i, j] * model.x[i, j] for i in model.I for j in model.J),
        sense=minimize
    )
#Constraints
def demand_rule(model, i):
        return sum(model.x[i, j] for j in model.J) == d[i]
model.demand = Constraint(model.I, rule=demand_rule)

def capacity_rule(model, j):
  return sum(model.x[i, j] for i in model.I) <= M[j] * model.y[j]
model.capacity = Constraint(model.J, rule=capacity_rule)

def strong_rule(model, i, j):
      return model.x[i, j] <= d[i] * model.y[j]
model.strong = Constraint(model.I, model.J, rule=strong_rule)

In [5]:
solver = SolverFactory('glpk')
solver.solve(model)

{'Problem': [{'Name': 'unknown', 'Lower bound': 5610.0, 'Upper bound': 5610.0, 'Number of objectives': 1, 'Number of constraints': 23, 'Number of variables': 18, 'Number of nonzeros': 63, 'Sense': 'minimize'}], 'Solver': [{'Status': 'ok', 'Termination condition': 'optimal', 'Statistics': {'Branch and bound': {'Number of bounded subproblems': '1', 'Number of created subproblems': '1'}}, 'Error rc': 0, 'Time': 0.004179954528808594}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [6]:
print("Objective Value:", model.obj())

Objective Value: 5610.0


In [7]:
EPS = 1e-5  # Define a small positive number
facilities = [j for j in model.y if model.y[j].value > EPS]
print("Facilities at Nodes:", facilities)

Facilities at Nodes: [2, 3]


In [8]:
edges = [(i,j) for (i,j) in model.x if model.x[i,j].value > EPS]
print("Edges:", edges)

Edges: [(1, 2), (2, 2), (3, 2), (3, 3), (4, 3), (5, 3)]
